### BW42_better_to_use_public_attribute_not_private_one
#### 파이썬 클래스 애트리뷰트 가시성 : **Public or Private**        
- 비공개 필드
  - attribute 앞에 _ 두개를 붙이면 비공개 필드가 됨(.으로 attribute 접근 가능)    
  - 클래스 안에 있는 메서드에서 해당 필드에 직접 접근 가능
  - 클래스 외부에서 비공개 필드에 접근하면 예외 발생
  - 하위 클래스는 부모 클래스의 비공개 필드에 접근 불가
  - **결론 : 비공개 attribute 는 하위 클래스의 필드와 이름이 충돌할 수 있는 경우만 써라**
- 보호 필드    
  - attribute 이름 앞에 _ 하나만 있으면 관례적으로 보호필드 의미
  - 조심히 쓰라는 의미

In [9]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
    
    def get_private_field(self):
        return self.__private_field

In [10]:
foo = MyObject()
assert foo.public_field == 5 # .연산자로 attribute 접근 가능

assert foo.get_private_field() == 10

In [11]:
# 클래스 외부에서는 비공개 필드에 접근하면 예외 발생
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

In [12]:
# 클래스 메서드는 해당 클래스의 비공개 필드에 접근 가능 1
#     - 자신을 둘러싸고 있는 class 블록 내부에 있기 때문에 
class MyOtherObject:
    def __init__(self):
        self.__private_field = 71
    
    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field

In [14]:
# 클래스 메서드는 해당 클래스의 비공개 필드에 접근 가능 2
bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 70

AssertionError: 

### 비공개 attribute에 대한 접근 구문이 실제로 엄격하게 제한 않는 이유는?
- **파이썬 모토 : 우리는 모두 책임질줄 아는 성인이다**    
  - 우리가 하고 싶은 일을 언어가 제한하면 안됨
  - 비공개 attribute 접근이 가능함으로써 얻는 이익이 해악보다 큼
  - attribute에 접근할 수 있는 언어 기능 제공 **BetterWay 47**    

In [15]:
# 하위 클래스는 부모 클래스의 비공개 필드에 접근 불가 1
# 자식 클래스에서 비공개 attribute에 접근할 경우, 
#       MychildObject__private_field의 이름으로 값을 찾음 -> 해당 변수 존재x 에러
class MyParentObject:
    def __init__(self):
        self.__private_field = 71
    
class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field
    
baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

In [16]:
# 하위 클래스는 부모 클래스의 비공개 필드에 접근 불가 2
# 아래의 방식으로 어디서든 원하는 클래스의 비공개 attribute에 접근 가능
assert baz._MyParentObject__private_field == 71

print(baz.__dict__)

{'_MyParentObject__private_field': 71}


In [24]:
# 비공개 attribute를 사용하게 되면 확장성이 떨어지고, 
#       오버라이드가 귀찮아지고 깨지기 쉬워짐    1
class MyStringClass:
    def __init__(self, value):
        self.__value = value
    
    def get_value(self):
        return str(self.__value)  
    
foo = MyStringClass(5)
assert foo.get_value() == '5' # 5가 맞으면 아무 에러 없음.. 틀리면 에러

class MyIntegerSubClass(MyStringClass):
    def get_value(self):
        return int(self._MyStringClass__value)
    
foo = MyIntegerSubClass('5')
assert foo.get_value() == 5

In [25]:
# 비공개 attribute를 사용하게 되면 확장성이 떨어지고, 
#       오버라이드가 귀찮아지고 깨지기 쉬워짐    2
class MyBaseClass:
    def __init__(self, value):
        self.__value = value
    
    def get_value(self):
        return self.__value
    
class MyStringClass(MyBaseClass):
    def get_value(self):
        return str(super().get_value()) # super() : 부모클래스 내용 사용할 경우
    
class MyIntegerSubClass(MyStringClass):
    def get_value(self):
        return int(self._MyStringClass__value)

In [27]:
# 상속을 허용하는 클래스들 사이에서 비공개 변수에 대한 참조가 깨짐 
#                  -> 보호 attribute를 사용하는 것이 더 나음
foo = MyIntegerSubClass(5)
foo.get_value()

AttributeError: 'MyIntegerSubClass' object has no attribute '_MyStringClass__value'

### 결론 : 비공개 attribute 는 하위 클래스의 필드와 이름이 충돌할 수 있는 경우만 써라

### BW47 지연 계산 attribute가 필요하면 __getattr__, __getattribute__, __setattr__ 를 사용해라
**지연 계산 : 매개변수에 있는 수식을 그대로 넘겨서 함수에서 매개변수가 필요할 때 계산**    
- ex: r(f(g(x))) 의 함수가 있을 때, 통상적으로 g->f->r 이지만, 지연 계산은 r->f->g 임
- 불필요한 계산을 안하게 됨
- 성능 향상, 무한한 계산 가능하게 함
- 기존 방식에 비해 **간결하고 가독성**이 우수함
- 구현시 유의 사항 **참조투명성(referential transparancy)**
  - 언제 계산하든 결과가 동일해야함
  - 여러번 계산되나 안되나 코드의 다른 부분에 영향을 미치지 않아야함
  - 수식이 계산된 부분과 계산되지 않은 부분을 구별하여 실행엔진에서 처리할 수 있어야함..
- yield, 제너레이터를 이용한 파이프라이닝   
## **Far to go...**

In [28]:
# __getattr__ : 특별 메서드로 동적 기능 활용
class LazyRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattr__(self, name):
        value = f'{name}를 위한 값'
        setattr(self, name, value)
        return value

In [29]:
data = LazyRecord()
print('이전: ', data.__dict__)
print('foo: ', data.foo)
print('이후: ', data.__dict__)


이전:  {'exists': 5}
foo:  foo를 위한 값
이후:  {'exists': 5, 'foo': 'foo를 위한 값'}


In [31]:
class LoggingLazyRecord(LazyRecord):
    def __getattr__(self, name):
        print(f'* 호출: __getattr__({name!r}), '
             f'인스턴스 딕셔너리 채워 넣음')
        result = super().__getattr__(name)
        print(f'* 반환: {result!r}')
        return result
    
data = LoggingLazyRecord()
print('exists : ', data.exists)
print('첫 번째 foo : ', data.foo)
print('두 번째 foo : ', data.foo)

exists :  5
* 호출: __getattr__('foo'), 인스턴스 딕셔너리 채워 넣음
* 반환: 'foo를 위한 값'
첫 번째 foo :  foo를 위한 값
두 번째 foo :  foo를 위한 값
